In [12]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math

In [17]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [14]:
df = pd.read_csv('~/data/training_data.csv')
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [19]:
X = df_normal.iloc[:,2].values
X = hankel.fit(X,30,0.5)
X = X.T
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
radiis = []
weights=[]
centers=[]
topR=[]
for i in range(optimal_k):
    cluster_points = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(X)
    U,Sigma,VT = np.linalg.svd(cluster_points)
    V = VT.T
    cluster_points = np.matmul(cluster_points,V[:,:r])
    weight,center = meepc.fit(cluster_points)
    var1=np.square(cluster_points-center)
    var2=np.matmul(weight,var1.T)
    max_radius=np.amax(var2)
    radiis.append(max_radius)
radiis



4


[1.0004404448941016, 1.0000054705202335, 1.0328305771318984, 1.000978109343972]

In [77]:
X = df.iloc[:,1].values
X = hankel.fit(X,30,0.5)
# print(X.shape)
r = rank.fit(X)
print(r)
U,Sigma,VT = np.linalg.svd(X)
# print(U.shape)
V = VT.T
X = np.matmul(X.T,U[:,:r])
weight,center = meepc.fit(X)
var1=np.square(X-center)
var2=np.matmul(weight,var1.T)
max_radius = math.sqrt(np.amax(var2))
max_radius


6


1.000003212252243

In [78]:
df_test = pd.read_csv('~/data/testdata.csv')
df_test = df_test.iloc[:, sensors]
Xtest = df_test.iloc[:,1].values

In [86]:
X_test = hankel.fit(Xtest,30,0.5)
X_test = X_test[:,19]
X_test = np.matmul(U[:,:r].T,X_test.reshape(-1,1))
# X_test = X_test.T
# print(X_test.shape)
weight,center = meepc.fit(X_test)
print(weight,center)
var1=np.square(X_test-center)
var2=np.matmul(weight,var1.T)
max_radius = math.sqrt(np.amax(var2))
max_radius

[0.00222796] [-21.20421687]


1.001713313658679